In [1]:
from sympy import symbols, diff, solve, N, Abs
import numpy as np
import pandas as pd

# =================================================================
# 1. MODELAGEM (POO com SymPy)
# =================================================================

class ModeloEOQ:
    """
    Implementa o modelo de Quantia Econômica de Pedido (EOQ)
    para minimizar o Custo Total de Estoque usando cálculo simbólico (SymPy).
    """

    def __init__(self, D: float, S: float, H: float):
        """
        Inicializa o modelo com os parâmetros do cliente (Persona Roberto).
        D: Demanda Anual, S: Custo por Pedido, H: Custo de Manutenção Anual.
        """
        if D <= 0 or S <= 0 or H <= 0:
            raise ValueError("Todos os parâmetros (D, S, H) devem ser positivos.")

        self.D = D
        self.S = S
        self.H = H

        # 1. Variável Simbólica: Q (Quantia do Pedido)
        self.Q = symbols('Q', positive=True)

        # 2. Função Custo Total Simbólica (C(Q))
        # C(Q) = (D*S)/Q + (Q*H)/2
        self.C_Q = (self.D * self.S) / self.Q + (self.Q * self.H) / 2
        
        # 3. Derivadas Simbólicas (Para justificativa no relatório)
        self.C_prime = diff(self.C_Q, self.Q)
        self.C_segunda = diff(self.C_prime, self.Q)


    def calcular_q_otimo(self) -> float:
        """
        Aplica o Critério da Primeira Derivada (C'(Q) = 0) para encontrar Q*.
        SymPy garante a precisão do cálculo.
        """
        # Resolve a equação C'(Q) = 0
        solucoes = solve(self.C_prime, self.Q)
        
        # O modelo EOQ tem apenas uma raiz positiva (Q*).
        # Usamos N() para obter o valor numérico com alta precisão.
        # Abs() é usado para garantir que pegamos apenas o valor positivo
        q_star_simbolico = Abs(solucoes[0])
        
        # Validação da Segunda Derivada (Garantir que é um MÍNIMO)
        # C''(Q) = 2*D*S / Q^3. Se > 0, é mínimo.
        cs_o_check = N(self.C_segunda.subs(self.Q, q_star_simbolico))
        if cs_o_check <= 0:
             print("Aviso: A segunda derivada não confirmou um mínimo. Verifique a modelagem.")
             
        return N(q_star_simbolico)


    def calcular_custo_total(self, Q_valor: float) -> float:
        """
        Calcula o custo total anual (C(Q)) para uma quantidade de pedido Q_valor.
        """
        # Substitui o valor de Q na função simbólica C(Q)
        custo_simbolico = self.C_Q.subs(self.Q, Q_valor)
        
        # Retorna o valor numérico
        return N(custo_simbolico)

# =================================================================
# 2. SIMULAÇÃO E COMPARAÇÃO (Testes e Avaliação)
# =================================================================

# 1. Dados da Persona (Roberto e o produto Pneu X200)
D_PNEU = 1000  # Demanda Anual (unidades)
S_PNEU = 100   # Custo por Pedido (R$)
H_PNEU = 5     # Custo de Manutenção Anual por unidade (R$)

# 2. Criação do Objeto Modelo
modelo = ModeloEOQ(D=D_PNEU, S=S_PNEU, H=H_PNEU)

# --- Cenário 1: Otimização ---
Q_otimo = modelo.calcular_q_otimo()
C_otimo = modelo.calcular_custo_total(Q_otimo)

# --- Cenário 2: Política Atual do Vendedor (Item a ser Comparado) ---
# Roberto atualmente faz pedidos de 150 unidades por chute.
Q_atual = 150 
C_atual = modelo.calcular_custo_total(Q_atual)

# --- Cenário 3: Política Alternativa (Exemplo de Cenário) ---
# Roberto pensa em fazer pedidos pequenos e frequentes.
Q_alternativo = 50
C_alternativo = modelo.calcular_custo_total(Q_alternativo)

# =================================================================
# 3. RESULTADO E APRESENTAÇÃO (Exibição para o Front End)
# =================================================================

print("="*50)
print("             SISTEMA DE OTIMIZAÇÃO DE ESTOQUE (EOQ)         ")
print("="*50)
print(f"Parâmetros de Entrada (Roberto): D={D_PNEU}, S={S_PNEU}, H={H_PNEU}")
print("-" * 50)

# 3a. Exibição da Solução Ótima (Q*)
print("\n### 🟢 RECOMENDAÇÃO ÓTIMA (CÁLCULO SYMPY) ###")
print(f"Quantia Ótima de Pedido (Q*): {Q_otimo:.2f} unidades")
print(f"Custo Total Mínimo (C(Q*)): R$ {C_otimo:.2f}")

# 3b. Comparação de Cenários
custos = [C_otimo, C_atual, C_alternativo]
quantidades = [Q_otimo, Q_atual, Q_alternativo]
titulos = ["Q* Ótimo", "Q Atual (150)", "Q Alternativo (50)"]

dados = {
    'Cenário': titulos,
    'Q (Pedido)': [f'{q:.2f}' for q in quantidades],
    'Custo Total Anual (R$)': [f'{c:.2f}' for c in custos],
    'Economia vs. Atual (%)': [
        f'{((C_atual - c) / C_atual * 100):.2f}%' if i != 1 else '---'
        for i, c in enumerate(custos)
    ]
}

df_comparacao = pd.DataFrame(dados)

print("\n### 📊 COMPARAÇÃO DE POLÍTICAS DE ESTOQUE ###")
print(df_comparacao.to_string(index=False))

# 3c. Detalhe da Justificativa
economia_q_otimo = (C_atual - C_otimo) / C_atual * 100
print("\n--- JUSTIFICATIVA ---")
print(f"A política ótima gera uma economia de {economia_q_otimo:.2f}% (R$ {C_atual - C_otimo:.2f})")
print("Este é o único ponto onde o Custo de Pedido é igual ao Custo de Manutenção.")

             SISTEMA DE OTIMIZAÇÃO DE ESTOQUE (EOQ)         
Parâmetros de Entrada (Roberto): D=1000, S=100, H=5
--------------------------------------------------

### 🟢 RECOMENDAÇÃO ÓTIMA (CÁLCULO SYMPY) ###
Quantia Ótima de Pedido (Q*): 200.00 unidades
Custo Total Mínimo (C(Q*)): R$ 1000.00

### 📊 COMPARAÇÃO DE POLÍTICAS DE ESTOQUE ###
           Cenário Q (Pedido) Custo Total Anual (R$) Economia vs. Atual (%)
          Q* Ótimo     200.00                1000.00                  4.00%
     Q Atual (150)     150.00                1041.67                    ---
Q Alternativo (50)      50.00                2125.00               -104.00%

--- JUSTIFICATIVA ---
A política ótima gera uma economia de 4.00% (R$ 41.67)
Este é o único ponto onde o Custo de Pedido é igual ao Custo de Manutenção.


In [3]:
# =================================================================
# 2. SIMULAÇÃO E COMPARAÇÃO (Testes e Avaliação)
# =================================================================

# 1. Dados da Persona (Roberto e o produto Pneu X200)
D_PNEU = 1000  # Demanda Anual (unidades)
S_PNEU = 300  # Custo por Pedido (R$)
H_PNEU = 5  # Custo de Manutenção Anual por unidade (R$)

# 2. Criação do Objeto Modelo
modelo = ModeloEOQ(D=D_PNEU, S=S_PNEU, H=H_PNEU)

# --- Cenário 1: Otimização ---
Q_otimo = modelo.calcular_q_otimo()
C_otimo = modelo.calcular_custo_total(Q_otimo)

# --- Cenário 2: Política Atual do Vendedor (Item a ser Comparado) ---
# Roberto atualmente faz pedidos de 150 unidades por chute.
Q_atual = 150
C_atual = modelo.calcular_custo_total(Q_atual)

# --- Cenário 3: Política Alternativa (Exemplo de Cenário) ---
# Roberto pensa em fazer pedidos pequenos e frequentes.
Q_alternativo = 50
C_alternativo = modelo.calcular_custo_total(Q_alternativo)

# =================================================================
# 3. RESULTADO E APRESENTAÇÃO (Exibição para o Front End)
# =================================================================

print("=" * 50)
print("             SISTEMA DE OTIMIZAÇÃO DE ESTOQUE (EOQ)         ")
print("=" * 50)
print(f"Parâmetros de Entrada (Roberto): D={D_PNEU}, S={S_PNEU}, H={H_PNEU}")
print("-" * 50)

# 3a. Exibição da Solução Ótima (Q*)
print("\n### 🟢 RECOMENDAÇÃO ÓTIMA (CÁLCULO SYMPY) ###")
print(f"Quantia Ótima de Pedido (Q*): {Q_otimo:.2f} unidades")
print(f"Custo Total Mínimo (C(Q*)): R$ {C_otimo:.2f}")

# 3b. Comparação de Cenários
custos = [C_otimo, C_atual, C_alternativo]
quantidades = [Q_otimo, Q_atual, Q_alternativo]
titulos = ["Q* Ótimo", "Q Atual (150)", "Q Alternativo (50)"]

dados = {
    "Cenário": titulos,
    "Q (Pedido)": [f"{q:.2f}" for q in quantidades],
    "Custo Total Anual (R$)": [f"{c:.2f}" for c in custos],
    "Economia vs. Atual (%)": [
        f"{((C_atual - c) / C_atual * 100):.2f}%" if i != 1 else "---"
        for i, c in enumerate(custos)
    ],
}

df_comparacao = pd.DataFrame(dados)

print("\n### 📊 COMPARAÇÃO DE POLÍTICAS DE ESTOQUE ###")
print(df_comparacao.to_string(index=False))

# 3c. Detalhe da Justificativa
economia_q_otimo = (C_atual - C_otimo) / C_atual * 100
print("\n--- JUSTIFICATIVA ---")
print(
    f"A política ótima gera uma economia de {economia_q_otimo:.2f}% (R$ {C_atual - C_otimo:.2f})"
)
print("Este é o único ponto onde o Custo de Pedido é igual ao Custo de Manutenção.")

             SISTEMA DE OTIMIZAÇÃO DE ESTOQUE (EOQ)         
Parâmetros de Entrada (Roberto): D=1000, S=300, H=5
--------------------------------------------------

### 🟢 RECOMENDAÇÃO ÓTIMA (CÁLCULO SYMPY) ###
Quantia Ótima de Pedido (Q*): 346.41 unidades
Custo Total Mínimo (C(Q*)): R$ 1732.05

### 📊 COMPARAÇÃO DE POLÍTICAS DE ESTOQUE ###
           Cenário Q (Pedido) Custo Total Anual (R$) Economia vs. Atual (%)
          Q* Ótimo     346.41                1732.05                 27.07%
     Q Atual (150)     150.00                2375.00                    ---
Q Alternativo (50)      50.00                6125.00               -157.89%

--- JUSTIFICATIVA ---
A política ótima gera uma economia de 27.07% (R$ 642.95)
Este é o único ponto onde o Custo de Pedido é igual ao Custo de Manutenção.
